In [ ]:
!pip install h5py

In [ ]:
import h5py
import os
train_data = r'train.h5'
db = h5py.File(train_data, 'r')


In [2]:
import tensorflow as tf
from tensorflow.keras import layers, models

def create_character_cnn(input_shape, num_classes):
    model = models.Sequential()
    
    # Convolutional layers
    model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(128, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    
    # Flatten layer to transition from convolutional to dense layers
    model.add(layers.Flatten())
    
    # Dense layers for classification
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dropout(0.5))  # Optional dropout layer for regularization
    model.add(layers.Dense(num_classes, activation='softmax'))
    
    # Compile the model
    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    
    return model

char_a_model = create_character_cnn(input_shape=(64, 64, 3), num_classes=7)

# Display model summaries
char_a_model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 62, 62, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 31, 31, 32)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 29, 29, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 14, 14, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 12, 12, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 6, 6, 128)         0

In [1]:


im_names = list(db['data'].keys())
x_train_masked_a = []
y_train_a = []
#mask images before feeding:
for name in im_names[:100]:
    img = db['data'][name][:]
    font = db['data'][name].attrs['font']
    txt = db['data'][name].attrs['txt']
    charBB = db['data'][name].attrs['charBB']
    wordBB = db['data'][name].attrs['wordBB']
    for w in range(wordBB.shape[-1]):
        cropped = crop_image(img, wordBB[:, :, w])
        if cropped is not None and np.any(cropped.size) and np.all(cropped.shape):
            x_train_masked_a.append(cropped)
            y_train_masked_a.append(font[w])
print('batch size = ', len(x_train_masked_a), len(y_train_masked_a))



ModuleNotFoundError: No module named 'utilities'

In [ ]:
# Example training with masked images
char_a_model.fit(x_train_masked_a, y_train_a, epochs=num_epochs, batch_size=batch_size)

# Example prediction with masked images
predictions_a = char_a_model.predict(x_test_masked_a)


This is how to submit

In [ ]:
# fonts_prediction_ids: [3,3,3,0,0,2,2 ...]
def submit_predictions(fonts_prediction_ids):
    
    df = pd.DataFrame(fonts_prediction_ids)
    df['Index'] = range(len(df))
    new_df = pd.DataFrame({'ind': df['Index'], 'font': fonts_prediction_ids}) 
    new_df.to_csv(_DIRNAME_WORKING_ + "submission.csv", index=False
map_dict = {0: 'Flower Rose Brush',
            1: 'Skylark',
            2: 'Sweet Puppy',
            3: 'Ubuntu Mono',
            4: 'VertigoFLF',
            5: 'Wanted M54',
            6: 'always forever'}

sub = pd.read_csv("sample_submission.csv")
sub['font'] = sub['font'].map({v: k for k, v in map_dict.items()})
new_df =pd.DataFrame({'ind': sub.index, 'font': sub['font'].values})
new_df.to_csv("new_sub.csv")



